# Tutorial on preparing train-test split for MORPH

Please save the test set id and the list of genes in the corresponding test set in `../data/{dataset_name}_splits.csv`

Example see `../data/example_data_splits.csv`

In [2]:
import pandas as pd
split_df = pd.read_csv('../data/example_data_splits.csv')
split_df

,data,test_set_id,test_set,note
0,example_data,random_split_1,"MAP4K5,KMT2A,UBASH3B+PTPN12,COL2A1,KLF1+BAK1,C...","random splits that includes 0/2, 1/2, 2/2 unse..."
